In [7]:
!pip install fastapi nest-asyncio pyngrok uvicorn


In [8]:
!ngrok authtoken 35bzLC1iSNO3dLdUNL5gSRQM4Do_2DSrmxDU4ETur3tb8CziU


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [9]:
import torch
from fastapi import FastAPI
from pydantic import BaseModel
import numpy as np
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import asyncio
import torch
import os
import torch.nn as nn

class TransformerClassifier(nn.Module):
    def __init__(self, input_size=2192, num_classes=102,
                 d_model=256, nhead=4, num_layers=2
                 , dim_feedforward=256, dropout=0.3):
        super().__init__()

        self.input_proj = nn.Sequential(
            nn.Linear(input_size, d_model),
            nn.Dropout(dropout)
        )

        self.pos_encoder = nn.Parameter(torch.randn(1, 60, d_model))
        self.dropout_pos = nn.Dropout(dropout)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead,
            dim_feedforward=dim_feedforward, dropout=dropout,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model,128),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.input_proj(x)                   # (B, T, d_model)
        x = x + self.pos_encoder[:, :x.size(1), :]
        x = self.dropout_pos(x)
        x = self.transformer(x)                  # (B, T, d_model)
        out = x.mean(dim=1)                      # Mean pooling
        out = self.fc(out)
        return out

# ---------------- LOAD MODEL ----------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerClassifier(input_size=2192, num_classes=102).to(device)
model.load_state_dict(torch.load("/content/256_256_128_102class_0.62.pth", map_location=device))
model.eval()

TransformerClassifier(
  (input_proj): Sequential(
    (0): Linear(in_features=2192, out_features=256, bias=True)
    (1): Dropout(p=0.3, inplace=False)
  )
  (dropout_pos): Dropout(p=0.3, inplace=False)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(in_features=256, out_features=256, bias=True)
        (dropout): Dropout(p=0.3, inplace=False)
        (linear2): Linear(in_features=256, out_features=256, bias=True)
        (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.3, inplace=False)
        (dropout2): Dropout(p=0.3, inplace=False)
      )
    )
  )
  (fc): Sequential(
    (0): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    

In [10]:
# state = torch.load("/content/256_256_128_2layers_0.78acc.pth")

# for k, v in state.items():
#     print(k, v.shape)


In [11]:
!pip install -q mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.


In [12]:
# !pip install numpy==1.26.4 --force-reinstall --no-cache-dir

In [13]:
import cv2
import numpy as np
import torch
from torchvision import models, transforms
import mediapipe as mp
from fastapi import FastAPI
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import asyncio
import base64
from scipy.interpolate import interp1d

# ---------------- Mediapipe setup ----------------
mp_holistic = mp.solutions.holistic
POSE_LANDMARKS = [
    mp.solutions.holistic.PoseLandmark.LEFT_SHOULDER,
    mp.solutions.holistic.PoseLandmark.LEFT_ELBOW,
    mp.solutions.holistic.PoseLandmark.LEFT_WRIST,
    mp.solutions.holistic.PoseLandmark.RIGHT_SHOULDER,
    mp.solutions.holistic.PoseLandmark.RIGHT_ELBOW,
    mp.solutions.holistic.PoseLandmark.RIGHT_WRIST,
]
N_POSE_LANDMARKS = len(POSE_LANDMARKS)
N_HAND_LANDMARKS = 21
N_TOTAL_LANDMARKS = N_POSE_LANDMARKS + 2 * N_HAND_LANDMARKS  # 48
POSE_FEATURES = N_TOTAL_LANDMARKS * 3  # 144

# ---------------- ResNet setup ----------------
device = "cuda" if torch.cuda.is_available() else "cpu"
resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1]).to(device)
resnet.eval()

resnet_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ---------------- Feature extraction ----------------
def extract_mediapipe_features_from_frame(frame):
    with mp_holistic.Holistic(static_image_mode=True) as holistic:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)

    pose_kps = np.zeros((N_POSE_LANDMARKS, 3))
    left_kps = np.zeros((N_HAND_LANDMARKS, 3))
    right_kps = np.zeros((N_HAND_LANDMARKS, 3))

    if results.pose_landmarks:
        for i, lm_id in enumerate(POSE_LANDMARKS):
            lm = results.pose_landmarks.landmark[lm_id]
            pose_kps[i] = [lm.x, lm.y, lm.z]

    if results.left_hand_landmarks:
        left_kps = np.array([[lm.x, lm.y, lm.z] for lm in results.left_hand_landmarks.landmark])

    if results.right_hand_landmarks:
        right_kps = np.array([[lm.x, lm.y, lm.z] for lm in results.right_hand_landmarks.landmark])

    return np.concatenate([pose_kps, left_kps, right_kps]).flatten()

@torch.no_grad()
def extract_resnet_features_from_frame(frame):
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    x = resnet_transform(img_rgb).unsqueeze(0).to(device)
    feat = resnet(x).squeeze().cpu().numpy()
    return feat

def extract_full_features(frame):
    kps = extract_mediapipe_features_from_frame(frame)
    cnn_feat = extract_resnet_features_from_frame(frame)
    return np.concatenate([kps, cnn_feat])  # 2192 vector



/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.7.1, so it will not be used.
  warnings.warn(


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 149MB/s]


In [14]:
# actions list
# ---------------- Base64 -> frame ----------------
def decode_base64_to_frame(base64_str):
    img_bytes = base64.b64decode(base64_str)
    np_arr = np.frombuffer(img_bytes, np.uint8)
    frame = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
    return frame

def interpolate_keypoints(keypoints_sequence, target_len=60):
    if len(keypoints_sequence) == 0:
        return None

    original_times = np.linspace(0, 1, len(keypoints_sequence))
    target_times = np.linspace(0, 1, target_len)
    num_features = keypoints_sequence[0].shape[0]

    interpolated_sequence = np.zeros((target_len, num_features))
    for i in range(num_features):
        feature_values = [frame[i] for frame in keypoints_sequence]
        interpolator = interp1d(original_times, feature_values,
                                kind='cubic', bounds_error=False,
                                fill_value='extrapolate')
        interpolated_sequence[:, i] = interpolator(target_times)

    return interpolated_sequence

In [15]:
import torch
import torch.nn as nn
import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI
import uvicorn
from pyngrok import ngrok
import asyncio
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware

# -----------------------
# Class mapping từ lúc train
# -----------------------
class_mapping = {'0': 0, '1': 1, '10': 2, '11': 3, '12': 4, '13': 5, '14': 6, '15': 7, '16': 8, '17': 9, '18': 10, '19': 11, '2': 12, '20': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, 'a': 21, 'anh_trai': 22, 'b': 23, 'bao_nhiêu': 24, 'buổi_chiều': 25, 'buổi_sáng': 26, 'buổi_trưa': 27, 'buổi_tối': 28, 'bà': 29, 'bác_sĩ': 30, 'bạn': 31, 'c': 32, 'cha': 33, 'chào': 34, 'chị_gái': 35, 'chồng': 36, 'còn_bạn': 37, 'công_an': 38, 'công_nhân': 39, 'cảm_ơn': 40, 'của_bạn': 41, 'của_tôi': 42, 'd': 43, 'dd': 44, 'dấu_móc': 45, 'dấu_mũ': 46, 'dấu_ă': 47, 'e': 48, 'ee': 49, 'em_gái': 50, 'em_trai': 51, 'g': 52, 'gia_đình': 53, 'giáo_viên': 54, 'h': 55, 'i': 56, 'k': 57, 'không': 58, 'kỹ_sư': 59, 'l': 60, 'm': 61, 'màu_cam': 62, 'màu_hồng': 63, 'màu_nâu': 64, 'màu_sắc': 65, 'màu_trắng': 66, 'màu_tím': 67, 'màu_xanh': 68, 'màu_đen': 69, 'màu_đỏ': 70, 'mẹ': 71, 'n': 72, 'nghề_nghiệp': 73, 'nhà': 74, 'o': 75, 'p': 76, 'phiên_dịch': 77, 'q': 78, 'r': 79, 's': 80, 't': 81, 'thích': 82, 'tuổi': 83, 'tên': 84, 'tôi': 85, 'tạm_biệt': 86, 'u': 87, 'v': 88, 'vợ': 89, 'x': 90, 'xin_lỗi': 91, 'y': 92, 'y_tá': 93, 'â': 94, 'ô': 95, 'ông': 96, 'ă': 97, 'đúng': 98, 'đúng_không': 99, 'ơ': 100, 'ư': 101}

# Tạo dict index -> label
index_to_label = {v: k for k, v in class_mapping.items()}

# -----------------------
# FastAPI
# -----------------------
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],      # Cho phép tất cả domain (frontend test)
    allow_methods=["*"],      # Cho phép tất cả HTTP methods
    allow_headers=["*"],      # Cho phép tất cả header
    allow_credentials=False   # Bắt buộc False nếu allow_origins="*"
)

class SequenceInput(BaseModel):
    data: list  # list các frame base64

@app.get("/")
def read_root():
    return {"message": "Hello World"}

@app.post("/predict")
def predict(input_data: SequenceInput):
    # Trích feature cho từng frame
    feature_list = []
    for b64 in input_data.data:
        frame = decode_base64_to_frame(b64)
        feat = extract_full_features(frame)  # 2192-dim
        feature_list.append(feat)

    feature_array = interpolate_keypoints(feature_list, target_len=60)  # (60, 2192)

    X = torch.tensor(feature_array, dtype=torch.float32).unsqueeze(0).to(device)  # (1, 60, 2192)

    with torch.no_grad():
        out = model(X)
        probs = torch.softmax(out, dim=1)
        pred_idx = torch.argmax(probs, dim=1).item()   # index
        confidence = probs.max().item()

    pred_label = index_to_label[pred_idx]  # label thật sự theo mapping lúc train

    return {"prediction": pred_label, "confidence": float(confidence)}

public_url = ngrok.connect(8000, bind_tls=True)
print(f"🌐 Public URL: {public_url}")

# Cấu hình server Uvicorn
config = uvicorn.Config(app=app, host="0.0.0.0", port=8000, log_level="info")
server = uvicorn.Server(config)

# Chạy server
asyncio.get_event_loop().create_task(server.serve())
print("✅ FastAPI server is running...")

🌐 Public URL: NgrokTunnel: "https://reasonably-protandrous-lexie.ngrok-free.dev" -> "http://localhost:8000"
✅ FastAPI server is running...
